# Python Depth Estimation With Stereo Vision

### This code is desgined to perform depth estimation on <span style="color:red">red</span> objects.

In [3]:
#Import necessary libraries and functions
import sys
import cv2
import numpy as np
import time
import imutils
from matplotlib import pyplot as plt

#Functions, will implement in different .py files
import HSV_filter as hsv
import shape_recognition as shape
import triangulation as tri

### For depth estimation to work, both a left and right camera must be used.

In [4]:
#Open both left and right cameras
cap_right = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cap_left = cv2.VideoCapture(1, cv2.CAP_DSHOW)

### We must also initialize some basic camera values.

In [5]:
frame_rate = 120 #Camera frame rate (Max 120 fps)

B = 9 #Distance between left and right camera [cm]
f = 6 #Camera Lens's focal length [cm]
alpha = 56.6 #Camera's field of view in the horizontal plane [degrees]


In [6]:
#Initial values
count = -1

while(True):
    count += 1
    
    #reading from both left and right camera
    ret_right, frame_right = cap_right.read() #ret is a boolean that returns whether or not there is an available frame
    ret_left, frame_left = cap_left.read()
    
    #If a frame cannot be captures from either the left or right camera, break the loop
    if ret_right == False or ret_left == False:
        break
        
    else:
        #APPLYING HSV-FILTER
        mask_right = hsv.add_HSV_filter(frame_right, 1)
        mask_left = hsv.add_HSV_filter(frame_left, 1)
        
        #Calculate resulting frames after applying HSV-filter mask
        res_right = cv2.bitwise_and(frame_right, frame_right, mask=mask_right) #segment the resulting right frame by bitwise anding the original right frame and its mask 
        res_left = cv2.bitwise_and(frame.left, frame_left, mask = mask_left) #segment the resulting right frame by bitwise anding the original left frame and its mask
        
        #APPLYING SHAPE RECOGNITION
        circles_right = shape.find_circles(frame_right, mask_right)
        circles_left = shape.find_circles(frame_left, mask_left)
        
        #If no ball/circle can be caught in the right or left camera, show text "TRACKING LOST"
        if np.all(circles_right) == None or np.all(circles_left == None):
            cv2.putText(frame_right, "TRACKING LOST", (75,50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
            cv2.putText(frame_left, "TRACKING LOST", (75,50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
            
        else:
            #Use triangulation to calculate object depth
            #Function to calculate depth of object. Outputs vetor of all depths in case of several balls/circles
            depth = tri.find_depth(circles_right, circles_left, frame_right, frame_left, B, f, alpha)
            
            cv2.putText(frame_right, "TRACKING", (75,50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (124,252,0), 2)
            cv2.putText(frame_left, "TRACKING", (75,50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (124,252,0), 2)
            cv2.putText(frame_right, "Distance: " + str(round(depth, 3)), (200,50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (124,252,0), 2)
            cv2.putText(frame_left, "Distance: " + str(round(depth, 3)), (200,50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (124,252,0), 2)
               
            print("Depth: ", depth)
            
        #Show frames
        cv2.imshow("frame right", frame_right)
        cv2.imshow("frame left", frame_left)
        cv2.imshow("mask right", mask_right)
        cv2.imshow("mask left", mask_left)

        if cv2.waitKey(1) & 0xFF == ord('q'): #Press q to close the window
            break
#Release and destroy all windows before termination
cap_right.release()
cap_left.release()

        